In [9]:
# rootの.envにOpenAIのAPIキーを配置し、pubcom_inputs配下にkankyou_pubcom.csvを配置して動かしてください

import pandas as pd
import json
import os

import os
import numpy as np
from pathlib import Path

import pandas as pd
import joblib
import sys
# kouchou-aiのパスを追加
sys.path.append("../kouchou-ai/server")
sys.path.append("../kouchou-ai/server/broadlistening/pipeline/")

In [10]:
input_dir = "../data/pubcom_inputs"
input_df = pd.read_csv(os.path.join(input_dir, "kankyou_pubcom_augmented.csv"))
embeddings = joblib.load(os.path.join(input_dir, "kankyou_pubcom_embeddings.pkl"))



In [31]:
exp_prefix = f"20250421_大量投稿実験_augmented"
OUTPUT_DIR = Path("../data/pubcom_outputs")
exp_output_dir = OUTPUT_DIR / exp_prefix

os.makedirs(exp_output_dir, exist_ok=True)

In [11]:
embeddings

array([[ 0.02302454, -0.00655467,  0.02373435, ...,  0.00896693,
        -0.00103214,  0.01552715],
       [ 0.04977854, -0.00664928, -0.01074246, ...,  0.00108663,
         0.03213061,  0.02791788],
       [ 0.06819215,  0.04907902,  0.02200207, ...,  0.0235509 ,
         0.00707406,  0.00135934],
       ...,
       [ 0.02092052,  0.00818217,  0.00177958, ..., -0.00365385,
        -0.02065316,  0.01894877],
       [ 0.02468635,  0.01152439,  0.05158343, ...,  0.0048513 ,
        -0.01722518,  0.01375558],
       [ 0.01658591, -0.0065666 ,  0.03142933, ...,  0.01077223,
        -0.01569315,  0.03127874]])

In [12]:
input_df

,要約,寄せられた御意見の概要,class,title,is_augmented
0,１ 福島の復興のため、復興再生利用等を推進すべき。,・ 福島県外での除染土の処分や再利用に賛成です。危険はありません。福島県の復興に是非邁進して...,1,福島の復興のため、復興再生利用等を推進すべき。,False
1,１ 福島の復興のため、復興再生利用等を推進すべき。,・ 大賛成！被災地の復興のために早急に実現して欲しい。科学的な判断に基づいて実行してください...,1,福島の復興のため、復興再生利用等を推進すべき。,False
2,１ 福島の復興のため、復興再生利用等を推進すべき。,・ 今回の省令改正は、福島の復興を推し進めるための除去土壌の再生利用を全国で円滑に進めるため...,1,福島の復興のため、復興再生利用等を推進すべき。,False
3,１ 福島の復興のため、復興再生利用等を推進すべき。,・ 今回の省令改正により、除去土壌の埋立処分を終了する場合の措置 、復興再生利用に用いる除去...,1,福島の復興のため、復興再生利用等を推進すべき。,False
4,１ 福島の復興のため、復興再生利用等を推進すべき。,・ 私は、福島の除去土壌を福島県外に移設することについては、止むを得ないと考えています。作業...,1,福島の復興のため、復興再生利用等を推進すべき。,False
...,...,...,...,...,...
442,13 除去土壌は全国にばら撒かず、発生地で集中管理すべき。,・ 汚染土の再利用は、地域ごとに負担の差が生じるため、不平等感が拡大する可能性があります。こ...,13,除去土壌は全国にばら撒かず、発生地で集中管理すべき。,True
443,13 除去土壌は全国にばら撒かず、発生地で集中管理すべき。,・ 汚染土を再利用する政策は、受け入れを強いられる地域にとって不公平感を生みやすく、社会の分...,13,除去土壌は全国にばら撒かず、発生地で集中管理すべき。,True
444,13 除去土壌は全国にばら撒かず、発生地で集中管理すべき。,・ 汚染土の再利用によって、特定の地域が不当に扱われていると感じることがあり、これが社会的な...,13,除去土壌は全国にばら撒かず、発生地で集中管理すべき。,True
445,13 除去土壌は全国にばら撒かず、発生地で集中管理すべき。,・ 汚染土の再利用政策は、地域間の不均衡を生み出しやすく、受け入れ地域の住民が不満を抱くこと...,13,除去土壌は全国にばら撒かず、発生地で集中管理すべき。,True


In [15]:
# 後のコードを流用するため変数名を変更
arguments_df = input_df.copy()
arguments_df["arg-id"] = arguments_df.index
arguments_df["argument"] = arguments_df["寄せられた御意見の概要"]


In [14]:
from umap import UMAP


cluster_nums = [77, 154]
n_samples = embeddings.shape[0]
default_n_neighbors = 15

if n_samples <= default_n_neighbors:
    n_neighbors = max(2, n_samples - 1)
else:
    n_neighbors = default_n_neighbors

umap_model = UMAP(random_state=42, n_components=2, n_neighbors=n_neighbors)
umap_embeds = umap_model.fit_transform(embeddings)


/Users/nasuka/.pyenv/versions/3.12.4/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/nasuka/.pyenv/versions/3.12.4/lib/python3.12/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


In [16]:
# 広聴AIで出力したレポートのslug
cluster_results = hierarchical_clustering_embeddings(
    umap_embeds=umap_embeds,
    cluster_nums=cluster_nums,
)
result_df = pd.DataFrame(
    {
        "arg-id": arguments_df["arg-id"],
        "argument": arguments_df["argument"],
        "x": umap_embeds[:, 0],
        "y": umap_embeds[:, 1],
    }
)
for cluster_level, final_labels in enumerate(cluster_results.values(), start=1):
    result_df[f"cluster-level-{cluster_level}-id"] = [f"{cluster_level}_{label}" for label in final_labels]



start initial clustering
end initial clustering
start hierarchical clustering
[77, 154]
n_cluster_cut:  77
end hierarchical clustering


In [23]:
from broadlistening.pipeline.steps.hierarchical_initial_labelling import initial_labelling

In [24]:
INITIAL_PROMPT = """あなたはKJ法が得意なデータ分析者です。userのinputはグループに集まったラベルです。なぜそのラベルが一つのグループであるか解説し、表札（label）をつけてください。
表札については、グループ内の具体的な論点や特徴を反映した、具体性の高い名称を考案してください。  
出力はJSONとし、フォーマットは以下のサンプルを参考にしてください。


# サンプルの入出力
## 入力例
- 手作業での意見分析は時間がかかりすぎる。AIで効率化できると嬉しい
- 今のやり方だと分析に工数がかかりすぎるけど、AIならコストをかけずに分析できそう
- AIが自動で意見を整理してくれると楽になって嬉しい


## 出力例
{{
    "label": "AIによる業務効率の大幅向上とコスト効率化",
    "description": "このクラスタは、従来の手作業による意見分析と比較して、AIによる自動化で分析プロセスが効率化され、作業時間の短縮や運用コストの効率化が実現される点に対する前向きな評価が中心です。"
}}"""

In [25]:
result_df

,arg-id,argument,x,y,cluster-level-1-id,cluster-level-2-id
0,0,・ 福島県外での除染土の処分や再利用に賛成です。危険はありません。福島県の復興に是非邁進して...,14.033040,7.198672,1_22,2_53
1,1,・ 大賛成！被災地の復興のために早急に実現して欲しい。科学的な判断に基づいて実行してください...,12.723913,7.349172,1_15,2_109
2,2,・ 今回の省令改正は、福島の復興を推し進めるための除去土壌の再生利用を全国で円滑に進めるため...,12.669956,7.381533,1_18,2_84
3,3,・ 今回の省令改正により、除去土壌の埋立処分を終了する場合の措置 、復興再生利用に用いる除去...,12.943524,7.536902,1_15,2_25
4,4,・ 私は、福島の除去土壌を福島県外に移設することについては、止むを得ないと考えています。作業...,13.866776,7.341768,1_21,2_94
...,...,...,...,...,...,...
442,442,・ 汚染土の再利用は、地域ごとに負担の差が生じるため、不平等感が拡大する可能性があります。こ...,14.609536,5.290862,1_31,2_97
443,443,・ 汚染土を再利用する政策は、受け入れを強いられる地域にとって不公平感を生みやすく、社会の分...,14.507370,5.437100,1_31,2_97
444,444,・ 汚染土の再利用によって、特定の地域が不当に扱われていると感じることがあり、これが社会的な...,14.676738,5.600533,1_31,2_134
445,445,・ 汚染土の再利用政策は、地域間の不均衡を生み出しやすく、受け入れ地域の住民が不満を抱くこと...,14.711903,5.498940,1_31,2_64


In [26]:
cluster_id_columns = [col for col in result_df.columns if col.startswith("cluster-level-")]
initial_cluster_id_column = cluster_id_columns[-1]
sampling_num = 30
initial_labelling_prompt = INITIAL_PROMPT
model = "gpt-4o"
workers = 70

initial_label_df = initial_labelling(
    initial_labelling_prompt,
    result_df,
    sampling_num,
    model,
    workers,
)

In [27]:
initial_label_df

,cluster_id,label,description
0,2_53,福島県外での除染土処分に関する意見の多様性と懸念,このクラスタは、福島県外での除染土の処分や再利用に関する意見の多様性を示しています。一部の意...
1,2_109,汚染土壌管理における責任の明確化と科学的アプローチの重要性,このクラスタは、福島第一原発事故に関連する汚染土壌の管理責任について、東京電力や国が主体的に...
2,2_84,福島復興における除去土壌再生利用の是非と国民理解の必要性,このクラスタは、福島の復興を目的とした除去土壌の再生利用に関する省令改正についての意見を集め...
3,2_25,福島復興に向けた除去土壌処分の全国的取り組みと国の主導,このクラスタは、福島の復興を促進するために、除去土壌の埋立処分や放射能濃度の基準、工事の施工...
4,2_94,福島除去土壌の処理に関する地域負担と責任分担の議論,このクラスタは、福島の除去土壌の処理に関する地域負担と責任分担についての意見が集まっています...
...,...,...,...
149,2_35,原子力発電所事故に関する法改正への市民意見集,このグループは、平成二十三年三月十一日に発生した東北地方太平洋沖地震に伴う原子力発電所事故に...
150,2_4,三原則の不遵守によるリスク管理の欠如,このクラスタは、特定の三原則（撤去、遮へい、距離を取る）に従っていないことに関する指摘が中心...
151,2_57,放射能汚染土の全国拡散に対する懸念と反対意見,このクラスタは、放射能で汚染された土壌を全国に運搬・拡散することに対する強い懸念と反対の意見...
152,2_8,放射性物質拡散による被ばくリスクと基本的人権の侵害,このクラスタは、放射能を含む土壌や残土を全国に拡散する計画に対する懸念を表しています。特に、...


In [28]:
initial_clusters_argument_df = result_df.merge(
    initial_label_df,
    left_on=initial_cluster_id_column,
    right_on="cluster_id",
    how="left",
).rename(
    columns={
        "label": f"{initial_cluster_id_column.replace('-id', '')}-label",
        "description": f"{initial_cluster_id_column.replace('-id', '')}-description",
    }
)

In [29]:
initial_clusters_argument_df

,arg-id,argument,x,y,cluster-level-1-id,cluster-level-2-id,cluster_id,cluster-level-2-label,cluster-level-2-description
0,0,・ 福島県外での除染土の処分や再利用に賛成です。危険はありません。福島県の復興に是非邁進して...,14.033040,7.198672,1_22,2_53,2_53,福島県外での除染土処分に関する意見の多様性と懸念,このクラスタは、福島県外での除染土の処分や再利用に関する意見の多様性を示しています。一部の意...
1,1,・ 大賛成！被災地の復興のために早急に実現して欲しい。科学的な判断に基づいて実行してください...,12.723913,7.349172,1_15,2_109,2_109,汚染土壌管理における責任の明確化と科学的アプローチの重要性,このクラスタは、福島第一原発事故に関連する汚染土壌の管理責任について、東京電力や国が主体的に...
2,2,・ 今回の省令改正は、福島の復興を推し進めるための除去土壌の再生利用を全国で円滑に進めるため...,12.669956,7.381533,1_18,2_84,2_84,福島復興における除去土壌再生利用の是非と国民理解の必要性,このクラスタは、福島の復興を目的とした除去土壌の再生利用に関する省令改正についての意見を集め...
3,3,・ 今回の省令改正により、除去土壌の埋立処分を終了する場合の措置 、復興再生利用に用いる除去...,12.943524,7.536902,1_15,2_25,2_25,福島復興に向けた除去土壌処分の全国的取り組みと国の主導,このクラスタは、福島の復興を促進するために、除去土壌の埋立処分や放射能濃度の基準、工事の施工...
4,4,・ 私は、福島の除去土壌を福島県外に移設することについては、止むを得ないと考えています。作業...,13.866776,7.341768,1_21,2_94,2_94,福島除去土壌の処理に関する地域負担と責任分担の議論,このクラスタは、福島の除去土壌の処理に関する地域負担と責任分担についての意見が集まっています...
...,...,...,...,...,...,...,...,...,...
442,442,・ 汚染土の再利用は、地域ごとに負担の差が生じるため、不平等感が拡大する可能性があります。こ...,14.609536,5.290862,1_31,2_97,2_97,汚染土再利用政策による地域間不平等と社会的リスク,このクラスタは、汚染土の再利用政策が地域間の不平等を助長し、社会的な対立や摩擦を引き起こすリ...
443,443,・ 汚染土を再利用する政策は、受け入れを強いられる地域にとって不公平感を生みやすく、社会の分...,14.507370,5.437100,1_31,2_97,2_97,汚染土再利用政策による地域間不平等と社会的リスク,このクラスタは、汚染土の再利用政策が地域間の不平等を助長し、社会的な対立や摩擦を引き起こすリ...
444,444,・ 汚染土の再利用によって、特定の地域が不当に扱われていると感じることがあり、これが社会的な...,14.676738,5.600533,1_31,2_134,2_134,汚染土再利用に伴う地域間の不公平感と社会的緊張,このクラスタは、汚染土の再利用に関する地域間の不公平感や不満が、社会的な緊張や対立を引き起こ...
445,445,・ 汚染土の再利用政策は、地域間の不均衡を生み出しやすく、受け入れ地域の住民が不満を抱くこと...,14.711903,5.498940,1_31,2_64,2_64,汚染土壌再利用による地域間不平等と社会的対立の懸念,このクラスタは、汚染された土壌の再利用が地域間での不平等感を生み出し、特定の地域が汚染を押し...


In [32]:
initial_clusters_argument_df.to_csv(exp_output_dir / "hierarchical_initial_labels.csv", index=False)

# 統合ラベリング


In [33]:
from broadlistening.pipeline.steps.hierarchical_merge_labelling import _filter_id_columns, merge_labelling, melt_cluster_data, _build_parent_child_mapping, calculate_cluster_density

In [34]:
MERGE_PROMPT = """あなたはデータ分析のエキスパートです。
現在、テキストデータの階層クラスタリングを行っています。
下層のクラスタのタイトルと説明、およびそれらのクラスタが所属する上層のクラスタのテキストのサンプルを与えるので、上層のクラスタのタイトルと説明を作成してください。

# 指示
- 統合後のクラスタ名は、統合前のクラスタ名称をそのまま引用せず、内容に基づいた新たな名称にしてください。  
- タイトルには、具体的な事象・行動（例：地域ごとの迅速対応、復興計画の着実な進展、効果的な情報共有・地域協力など）を含めてください
  - 可能な限り具体的な表現を用いるようにし、抽象的な表現は避けてください
    - 「多様な意見」などの抽象的な表現は避けてください
- 出力例に示したJSON形式で出力してください


# サンプルの入出力
## 入力例
- 「顧客フィードバックの自動集約」: このクラスタは、SNSやオンラインレビューなどから集めた大量の意見をAIが瞬時に解析し、企業が市場のトレンドや顧客の要望を即時に把握できる点についての期待を示しています。
- 「AIによる業務効率の大幅向上とコスト効率化」: このクラスタは、従来の手作業による意見分析と比較して、AIによる自動化で分析プロセスが効率化され、作業時間の短縮や運用コストの効率化が実現される点に対する前向きな評価が中心です。

## 出力例
{{
    "label": "AI技術の導入による意見分析の効率化への期待",
    "description": "大量の意見やフィードバックから迅速に洞察を抽出できるため、企業や自治体が消費者や市民の声を的確に把握し、戦略的な意思決定やサービス改善が可能になります。また、従来の手法と比べて作業負荷が軽減され、業務効率の向上やコスト削減といった実際の便益が得られると期待されています。"
}}"""

In [35]:
config = {}
config["hierarchical_merge_labelling"] = {
    "prompt": MERGE_PROMPT,
    "model": "gpt-4o",
    "sampling_num": 50,
    "workers": 70,
}

In [36]:
clusters_df = initial_clusters_argument_df.copy()
cluster_id_columns: list[str] = _filter_id_columns(clusters_df.columns)
merge_result_df = merge_labelling(
    clusters_df=clusters_df,
    cluster_id_columns=sorted(cluster_id_columns, reverse=True),
    config=config,
)

100%|██████████| 1/1 [00:09<00:00,  9.23s/it]


In [37]:
melted_df = melt_cluster_data(merge_result_df)
parent_child_df = _build_parent_child_mapping(merge_result_df, cluster_id_columns)
melted_df = melted_df.merge(parent_child_df, on=["level", "id"], how="left")
melted_df.head(20)

,level,id,label,description,value,parent
0,1,1_22,福島除染土の処分に関する地域的影響と法的・倫理的課題の再評価,福島県外での除染土の処分に関する多様な意見が存在し、地域社会への影響や法的・倫理的課題が浮き...,11,0
1,1,1_15,福島復興に向けた汚染土壌管理の責任明確化と科学的政策推進,福島第一原発事故に関連する汚染土壌の管理において、東京電力や国が主体的に責任を負うべきである...,5,0
2,1,1_18,福島復興における除去土壌再生利用と健康保護の調和,福島の復興を目指す除去土壌の再生利用に関する省令改正について、国民の健康保護と復興施策の進行...,4,0
3,1,1_21,福島除去土壌処理における地域間責任と政策形成の現場重視,福島の除去土壌処理に関する地域負担と責任分担の議論では、福島県内での集中管理を支持する意見や...,7,0
4,1,1_52,放射能汚染土再利用の安全性と教育アプローチの再考,放射能汚染土の再利用に関する政策は、安全性の確保と社会的受容性の両面で議論を呼んでいます。安...,4,0
5,1,1_40,政策・法案に対する強い反対意志の表明とその反復,特定の政策や法案に対する強い反対意志が、感情的かつ繰り返しの表現を通じて示されています。特に...,13,0
6,1,1_42,基準緩和に対する不満と透明性への要求,基準を大幅に緩和する決定に対して、理解不足やその意図の不透明さに対する強い不満が表明されてい...,2,0
7,1,1_25,汚染土再利用に対する安全性確保と意見収集の重要性,汚染土の再利用に関して、住民や作業者からの強い懸念が示されており、安全性を確保するための具体...,5,0
8,1,1_29,放射能汚染土の再利用に対する反対意見と利権批判,このクラスタは、放射能汚染土の再利用に対する強い反対意見と、それに関連する公共事業における利...,2,0
9,1,1_26,除染土再利用政策への不信と透明性確保の要求,除染土の再利用に関する政策に対して、安全性や透明性の欠如を指摘する意見が集まっています。特に...,6,0


In [38]:
melted_df.to_csv(exp_output_dir / "クラスタタイトル.csv", index=False)

In [39]:
merge_result_df

,arg-id,argument,x,y,cluster-level-1-id,cluster-level-2-id,cluster_id,cluster-level-2-label,cluster-level-2-description,cluster-level-1-label,cluster-level-1-description
0,0,・ 福島県外での除染土の処分や再利用に賛成です。危険はありません。福島県の復興に是非邁進して...,14.033040,7.198672,1_22,2_53,2_53,福島県外での除染土処分に関する意見の多様性と懸念,このクラスタは、福島県外での除染土の処分や再利用に関する意見の多様性を示しています。一部の意...,福島除染土の処分に関する地域的影響と法的・倫理的課題の再評価,福島県外での除染土の処分に関する多様な意見が存在し、地域社会への影響や法的・倫理的課題が浮き...
1,1,・ 大賛成！被災地の復興のために早急に実現して欲しい。科学的な判断に基づいて実行してください...,12.723913,7.349172,1_15,2_109,2_109,汚染土壌管理における責任の明確化と科学的アプローチの重要性,このクラスタは、福島第一原発事故に関連する汚染土壌の管理責任について、東京電力や国が主体的に...,福島復興に向けた汚染土壌管理の責任明確化と科学的政策推進,福島第一原発事故に関連する汚染土壌の管理において、東京電力や国が主体的に責任を負うべきである...
2,2,・ 今回の省令改正は、福島の復興を推し進めるための除去土壌の再生利用を全国で円滑に進めるため...,12.669956,7.381533,1_18,2_84,2_84,福島復興における除去土壌再生利用の是非と国民理解の必要性,このクラスタは、福島の復興を目的とした除去土壌の再生利用に関する省令改正についての意見を集め...,福島復興における除去土壌再生利用と健康保護の調和,福島の復興を目指す除去土壌の再生利用に関する省令改正について、国民の健康保護と復興施策の進行...
3,3,・ 今回の省令改正により、除去土壌の埋立処分を終了する場合の措置 、復興再生利用に用いる除去...,12.943524,7.536902,1_15,2_25,2_25,福島復興に向けた除去土壌処分の全国的取り組みと国の主導,このクラスタは、福島の復興を促進するために、除去土壌の埋立処分や放射能濃度の基準、工事の施工...,福島復興に向けた汚染土壌管理の責任明確化と科学的政策推進,福島第一原発事故に関連する汚染土壌の管理において、東京電力や国が主体的に責任を負うべきである...
4,4,・ 私は、福島の除去土壌を福島県外に移設することについては、止むを得ないと考えています。作業...,13.866776,7.341768,1_21,2_94,2_94,福島除去土壌の処理に関する地域負担と責任分担の議論,このクラスタは、福島の除去土壌の処理に関する地域負担と責任分担についての意見が集まっています...,福島除去土壌処理における地域間責任と政策形成の現場重視,福島の除去土壌処理に関する地域負担と責任分担の議論では、福島県内での集中管理を支持する意見や...
...,...,...,...,...,...,...,...,...,...,...,...
442,442,・ 汚染土の再利用は、地域ごとに負担の差が生じるため、不平等感が拡大する可能性があります。こ...,14.609536,5.290862,1_31,2_97,2_97,汚染土再利用政策による地域間不平等と社会的リスク,このクラスタは、汚染土の再利用政策が地域間の不平等を助長し、社会的な対立や摩擦を引き起こすリ...,汚染土再利用政策による地域間不平等と社会的対立のリスク管理,汚染土の再利用政策は、特定の地域に不公平な負担を強いることで、地域間の不平等感を助長し、社会...
443,443,・ 汚染土を再利用する政策は、受け入れを強いられる地域にとって不公平感を生みやすく、社会の分...,14.507370,5.437100,1_31,2_97,2_97,汚染土再利用政策による地域間不平等と社会的リスク,このクラスタは、汚染土の再利用政策が地域間の不平等を助長し、社会的な対立や摩擦を引き起こすリ...,汚染土再利用政策による地域間不平等と社会的対立のリスク管理,汚染土の再利用政策は、特定の地域に不公平な負担を強いることで、地域間の不平等感を助長し、社会...
444,444,・ 汚染土の再利用によって、特定の地域が不当に扱われていると感じることがあり、これが社会的な...,14.676738,5.600533,1_31,2_134,2_134,汚染土再利用に伴う地域間の不公平感と社会的緊張,このクラスタは、汚染土の再利用に関する地域間の不公平感や不満が、社会的な緊張や対立を引き起こ...,汚染土再利用政策による地域間不平等と社会的対立のリスク管理,汚染土の再利用政策は、特定の地域に不公平な負担を強いることで、地域間の不平等感を助長し、社会...
445,445,・ 汚染土の再利用政策は、地域間の不均衡を生み出しやすく、受け入れ地域の住民が不満を抱くこと...,14.711903,5.498940,1_31,2_64,2_64,汚染土壌再利用による地域間不平等と社会的対立の懸念,このクラスタは、汚染された土壌の再利用が地域間での不平等感を生み出し、特定の地域が汚染を押し...,汚染土再利用政策による地域間不平等と社会的対立のリスク管理,汚染土の再利用政策は、特定の地域に不公平な負担を強いることで、地域間の不平等感を助長し、社会...


In [41]:
arguments_df

,要約,寄せられた御意見の概要,class,title,is_augmented,arg-id,argument
0,１ 福島の復興のため、復興再生利用等を推進すべき。,・ 福島県外での除染土の処分や再利用に賛成です。危険はありません。福島県の復興に是非邁進して...,1,福島の復興のため、復興再生利用等を推進すべき。,False,0,・ 福島県外での除染土の処分や再利用に賛成です。危険はありません。福島県の復興に是非邁進して...
1,１ 福島の復興のため、復興再生利用等を推進すべき。,・ 大賛成！被災地の復興のために早急に実現して欲しい。科学的な判断に基づいて実行してください...,1,福島の復興のため、復興再生利用等を推進すべき。,False,1,・ 大賛成！被災地の復興のために早急に実現して欲しい。科学的な判断に基づいて実行してください...
2,１ 福島の復興のため、復興再生利用等を推進すべき。,・ 今回の省令改正は、福島の復興を推し進めるための除去土壌の再生利用を全国で円滑に進めるため...,1,福島の復興のため、復興再生利用等を推進すべき。,False,2,・ 今回の省令改正は、福島の復興を推し進めるための除去土壌の再生利用を全国で円滑に進めるため...
3,１ 福島の復興のため、復興再生利用等を推進すべき。,・ 今回の省令改正により、除去土壌の埋立処分を終了する場合の措置 、復興再生利用に用いる除去...,1,福島の復興のため、復興再生利用等を推進すべき。,False,3,・ 今回の省令改正により、除去土壌の埋立処分を終了する場合の措置 、復興再生利用に用いる除去...
4,１ 福島の復興のため、復興再生利用等を推進すべき。,・ 私は、福島の除去土壌を福島県外に移設することについては、止むを得ないと考えています。作業...,1,福島の復興のため、復興再生利用等を推進すべき。,False,4,・ 私は、福島の除去土壌を福島県外に移設することについては、止むを得ないと考えています。作業...
...,...,...,...,...,...,...,...
442,13 除去土壌は全国にばら撒かず、発生地で集中管理すべき。,・ 汚染土の再利用は、地域ごとに負担の差が生じるため、不平等感が拡大する可能性があります。こ...,13,除去土壌は全国にばら撒かず、発生地で集中管理すべき。,True,442,・ 汚染土の再利用は、地域ごとに負担の差が生じるため、不平等感が拡大する可能性があります。こ...
443,13 除去土壌は全国にばら撒かず、発生地で集中管理すべき。,・ 汚染土を再利用する政策は、受け入れを強いられる地域にとって不公平感を生みやすく、社会の分...,13,除去土壌は全国にばら撒かず、発生地で集中管理すべき。,True,443,・ 汚染土を再利用する政策は、受け入れを強いられる地域にとって不公平感を生みやすく、社会の分...
444,13 除去土壌は全国にばら撒かず、発生地で集中管理すべき。,・ 汚染土の再利用によって、特定の地域が不当に扱われていると感じることがあり、これが社会的な...,13,除去土壌は全国にばら撒かず、発生地で集中管理すべき。,True,444,・ 汚染土の再利用によって、特定の地域が不当に扱われていると感じることがあり、これが社会的な...
445,13 除去土壌は全国にばら撒かず、発生地で集中管理すべき。,・ 汚染土の再利用政策は、地域間の不均衡を生み出しやすく、受け入れ地域の住民が不満を抱くこと...,13,除去土壌は全国にばら撒かず、発生地で集中管理すべき。,True,445,・ 汚染土の再利用政策は、地域間の不均衡を生み出しやすく、受け入れ地域の住民が不満を抱くこと...


In [40]:
result_df

,arg-id,argument,x,y,cluster-level-1-id,cluster-level-2-id
0,0,・ 福島県外での除染土の処分や再利用に賛成です。危険はありません。福島県の復興に是非邁進して...,14.033040,7.198672,1_22,2_53
1,1,・ 大賛成！被災地の復興のために早急に実現して欲しい。科学的な判断に基づいて実行してください...,12.723913,7.349172,1_15,2_109
2,2,・ 今回の省令改正は、福島の復興を推し進めるための除去土壌の再生利用を全国で円滑に進めるため...,12.669956,7.381533,1_18,2_84
3,3,・ 今回の省令改正により、除去土壌の埋立処分を終了する場合の措置 、復興再生利用に用いる除去...,12.943524,7.536902,1_15,2_25
4,4,・ 私は、福島の除去土壌を福島県外に移設することについては、止むを得ないと考えています。作業...,13.866776,7.341768,1_21,2_94
...,...,...,...,...,...,...
442,442,・ 汚染土の再利用は、地域ごとに負担の差が生じるため、不平等感が拡大する可能性があります。こ...,14.609536,5.290862,1_31,2_97
443,443,・ 汚染土を再利用する政策は、受け入れを強いられる地域にとって不公平感を生みやすく、社会の分...,14.507370,5.437100,1_31,2_97
444,444,・ 汚染土の再利用によって、特定の地域が不当に扱われていると感じることがあり、これが社会的な...,14.676738,5.600533,1_31,2_134
445,445,・ 汚染土の再利用政策は、地域間の不均衡を生み出しやすく、受け入れ地域の住民が不満を抱くこと...,14.711903,5.498940,1_31,2_64


In [54]:
output_df = merge_result_df.merge(arguments_df[["arg-id", "class", "title", "is_augmented"]], on="arg-id", how="left").rename({
    "class": "元クラス",
    "title": "元タイトル",
    "cluster-level-1-id": "広聴AIクラスタid",
    "cluster-level-2-label": "広聴AIクラスタラベル",
    "cluster-level-2-description": "広聴AIクラスタ説明",
},
axis=1
)

In [58]:
output_filename =  f"クラスタリングラベル_{exp_prefix}.csv"
output_cols = ["arg-id",  "元クラス", "元タイトル", "is_augmented", "広聴AIクラスタid", "広聴AIクラスタラベル", "広聴AIクラスタ説明",]
output_df[output_cols].to_csv(exp_output_dir / output_filename, index=False)
output_df[output_cols]

,arg-id,元クラス,元タイトル,is_augmented,広聴AIクラスタid,広聴AIクラスタラベル,広聴AIクラスタ説明
0,0,1,福島の復興のため、復興再生利用等を推進すべき。,False,1_22,福島県外での除染土処分に関する意見の多様性と懸念,このクラスタは、福島県外での除染土の処分や再利用に関する意見の多様性を示しています。一部の意...
1,1,1,福島の復興のため、復興再生利用等を推進すべき。,False,1_15,汚染土壌管理における責任の明確化と科学的アプローチの重要性,このクラスタは、福島第一原発事故に関連する汚染土壌の管理責任について、東京電力や国が主体的に...
2,2,1,福島の復興のため、復興再生利用等を推進すべき。,False,1_18,福島復興における除去土壌再生利用の是非と国民理解の必要性,このクラスタは、福島の復興を目的とした除去土壌の再生利用に関する省令改正についての意見を集め...
3,3,1,福島の復興のため、復興再生利用等を推進すべき。,False,1_15,福島復興に向けた除去土壌処分の全国的取り組みと国の主導,このクラスタは、福島の復興を促進するために、除去土壌の埋立処分や放射能濃度の基準、工事の施工...
4,4,1,福島の復興のため、復興再生利用等を推進すべき。,False,1_21,福島除去土壌の処理に関する地域負担と責任分担の議論,このクラスタは、福島の除去土壌の処理に関する地域負担と責任分担についての意見が集まっています...
...,...,...,...,...,...,...,...
442,442,13,除去土壌は全国にばら撒かず、発生地で集中管理すべき。,True,1_31,汚染土再利用政策による地域間不平等と社会的リスク,このクラスタは、汚染土の再利用政策が地域間の不平等を助長し、社会的な対立や摩擦を引き起こすリ...
443,443,13,除去土壌は全国にばら撒かず、発生地で集中管理すべき。,True,1_31,汚染土再利用政策による地域間不平等と社会的リスク,このクラスタは、汚染土の再利用政策が地域間の不平等を助長し、社会的な対立や摩擦を引き起こすリ...
444,444,13,除去土壌は全国にばら撒かず、発生地で集中管理すべき。,True,1_31,汚染土再利用に伴う地域間の不公平感と社会的緊張,このクラスタは、汚染土の再利用に関する地域間の不公平感や不満が、社会的な緊張や対立を引き起こ...
445,445,13,除去土壌は全国にばら撒かず、発生地で集中管理すべき。,True,1_31,汚染土壌再利用による地域間不平等と社会的対立の懸念,このクラスタは、汚染された土壌の再利用が地域間での不平等感を生み出し、特定の地域が汚染を押し...


In [61]:
output_df[output_cols].head(20)

,arg-id,元クラス,元タイトル,is_augmented,広聴AIクラスタid,広聴AIクラスタラベル,広聴AIクラスタ説明
0,0,1,福島の復興のため、復興再生利用等を推進すべき。,False,1_22,福島県外での除染土処分に関する意見の多様性と懸念,このクラスタは、福島県外での除染土の処分や再利用に関する意見の多様性を示しています。一部の意...
1,1,1,福島の復興のため、復興再生利用等を推進すべき。,False,1_15,汚染土壌管理における責任の明確化と科学的アプローチの重要性,このクラスタは、福島第一原発事故に関連する汚染土壌の管理責任について、東京電力や国が主体的に...
2,2,1,福島の復興のため、復興再生利用等を推進すべき。,False,1_18,福島復興における除去土壌再生利用の是非と国民理解の必要性,このクラスタは、福島の復興を目的とした除去土壌の再生利用に関する省令改正についての意見を集め...
3,3,1,福島の復興のため、復興再生利用等を推進すべき。,False,1_15,福島復興に向けた除去土壌処分の全国的取り組みと国の主導,このクラスタは、福島の復興を促進するために、除去土壌の埋立処分や放射能濃度の基準、工事の施工...
4,4,1,福島の復興のため、復興再生利用等を推進すべき。,False,1_21,福島除去土壌の処理に関する地域負担と責任分担の議論,このクラスタは、福島の除去土壌の処理に関する地域負担と責任分担についての意見が集まっています...
5,5,1,福島の復興のため、復興再生利用等を推進すべき。,False,1_22,福島除去土壌の最終処分に関する法的・倫理的課題と地域影響,このクラスタは、福島の除去土壌の最終処分に関する法的な約束やその実行可能性、地域社会への影響...
6,6,1,福島の復興のため、復興再生利用等を推進すべき。,False,1_52,放射能汚染土の再利用に関する安全性と社会的影響の対立,このクラスタは、放射能汚染土の再利用に関する安全性とその社会的影響についての意見が対立してい...
7,7,2,反対。（理由の記載なし）,False,1_40,強い反対の意思表示とその反復,このクラスタは、特定の政策や提案に対する強い反対の意思を示す意見で構成されています。意見の多...
8,8,2,反対。（理由の記載なし）,False,1_40,法案への否定的意見,このクラスタは、特定の法案に対する否定的な意見や反対の立場を示す発言で構成されています。短い...
9,9,2,反対。（理由の記載なし）,False,1_40,強い反対の意思表示とその反復,このクラスタは、特定の政策や提案に対する強い反対の意思を示す意見で構成されています。意見の多...
